In [18]:
!wget http://www.manythings.org/anki/hin-eng.zip
!unzip hin-eng.zip

--2019-12-17 13:19:50--  http://www.manythings.org/anki/hin-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.108.196, 104.24.109.196, 2606:4700:30::6818:6dc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.108.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 126760 (124K) [application/zip]
Saving to: ‘hin-eng.zip.2’

hin-eng.zip.2       100%[===================>] 123.79K   178KB/s    in 0.7s    

2019-12-17 13:19:51 (178 KB/s) - ‘hin-eng.zip.2’ saved [126760/126760]

Archive:  hin-eng.zip
replace hin.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: hin.txt                 
replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: _about.txt              


In [1]:
import os
os.listdir()

['.config', 'hin-eng.zip', '_about.txt', 'hin.txt', 'sample_data']

In [2]:
from __future__ import print_function

import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

print(f"keras version: {keras.__version__}")

Using TensorFlow backend.


keras version: 2.2.5


In [0]:
batch_size = 500  # Batch size for training.
epochs = 25  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'hin.txt'


In [0]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')


In [5]:
lines[100:105]

["I don't know.\tमुझे नहीं पता।\tCC-BY 2.0 (France) Attribution: tatoeba.org #349064 (fatih) & #609376 (minshirui)",
 "I don't know.\tमुझे नहीं मालूम।\tCC-BY 2.0 (France) Attribution: tatoeba.org #349064 (fatih) & #609377 (minshirui)",
 'I have a car.\tमेरे पास एक गाड़ी है।\tCC-BY 2.0 (France) Attribution: tatoeba.org #252272 (CK) & #477720 (minshirui)',
 'I have a dog.\tमेरे पास एक कुत्ता है।\tCC-BY 2.0 (France) Attribution: tatoeba.org #378502 (CK) & #443037 (minshirui)',
 'I understand.\tमैं समझता हूँ।\tCC-BY 2.0 (France) Attribution: tatoeba.org #433468 (CK) & #588495 (minshirui)']

In [0]:
# Vectorize the data.
input_texts = []
target_texts = []
input_chars = set()
target_chars = set()

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)

    # add all characters in the buckets, named input_chars, target_chars
    for char in input_text:
        if char not in input_chars:
            input_chars.add(char)
    for char in target_text:
        if char not in target_chars:
            target_chars.add(char)

In [7]:
input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))
n_enc_tokens = len(input_chars)
n_dec_tokens = len(target_chars)
enc_seq_len = max([len(txt) for txt in input_texts])
dec_seq_len = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', n_enc_tokens)
print('Number of unique output tokens:', n_dec_tokens)
print('Max sequence length for inputs:', enc_seq_len)
print('Max sequence length for outputs:', dec_seq_len)

Number of samples: 2785
Number of unique input tokens: 70
Number of unique output tokens: 92
Max sequence length for inputs: 107
Max sequence length for outputs: 123


In [0]:
input_token_dict = dict([(char, i) for i, char in enumerate(input_chars)])
target_token_dict = dict([(char, i) for i, char in enumerate(target_chars)])

In [0]:
enc_input_data  = np.zeros((len(input_texts), enc_seq_len, n_enc_tokens), dtype='float32')
dec_input_data  = np.zeros((len(input_texts), dec_seq_len, n_dec_tokens), dtype='float32')
dec_target_data = np.zeros((len(input_texts), dec_seq_len, n_dec_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        enc_input_data[i, t, input_token_dict[char]] = 1.
    # fill the rest of sequence with spaces
    enc_input_data[i, t + 1:, input_token_dict[' ']] = 1.

    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        dec_input_data[i, t, target_token_dict[char]] = 1.

        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            dec_target_data[i, t - 1, target_token_dict[char]] = 1.
    # fill the rest of sequence with spaces
    dec_input_data[i, t + 1:, target_token_dict[' ']] = 1.
    dec_target_data[i, t:, target_token_dict[' ']] = 1.


In [10]:
np.argmax(enc_input_data[0], axis=1).ravel()

array([42, 58, 66,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0])

In [0]:
# from keras.layers import Dense, Flatten, LSTM, Activation
# from keras.layers import Dropout, RepeatVector, TimeDistributed
# from keras import Input, Model

# seq_length = 15
# input_dims = 10
# output_dims = 8 # number of classes
# n_hidden = 10
# model1_inputs = Input(shape=(seq_length,input_dims,))
# model1_outputs = Input(shape=(output_dims,))

# net1 = LSTM(n_hidden, return_sequences=True)(model1_inputs)
# net1 = LSTM(n_hidden, return_sequences=False)(net1)
# net1 = Dense(output_dims, activation='relu')(net1)
# model1_outputs = net1

# model1 = Model(inputs=model1_inputs, outputs = model1_outputs, name='model1')

# ## Fit the model
# model1.summary()


# _________________________________________________________________
# Layer (type)                 Output Shape              Param #   
# =================================================================
# input_1 (InputLayer)        (None, 15, 10)            0         
# _________________________________________________________________
# lstm_1 (LSTM)                (None, 15, 10)            840       
# _________________________________________________________________
# lstm_2 (LSTM)                (None, 10)                840       
# _________________________________________________________________
# dense_3 (Dense)              (None, 8)                 88        
# _________________________________________________________________

In [13]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, n_enc_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, n_dec_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(n_dec_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 2
model.fit(
    [enc_input_data, dec_input_data], 
    dec_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2
)
# Save model
model.save('eng2hin.h5')



Train on 2228 samples, validate on 557 samples
Epoch 1/2
2228/2228 [==============================] - 33s 15ms/step - loss: 2.6686 - acc: 0.6259 - val_loss: 1.7741 - val_acc: 0.6863
Epoch 2/2
2228/2228 [==============================] - 31s 14ms/step - loss: 1.0392 - acc: 0.8070 - val_loss: 1.5440 - val_acc: 0.6883


In [16]:
for i in range(5):
    print("="*50)
    model.fit(
        [enc_input_data, dec_input_data], 
        dec_target_data,
        batch_size=batch_size,
        epochs=1,
        validation_split=0.2
    )

    for seq_index in range(5):
        # Take one sequence (part of the training set)
        # for trying out decoding.
        input_seq = enc_input_data[seq_index: seq_index + 1]
        decoded_sentence = decode_sequence(input_seq)
        print('-'*50)
        print('Input sentence:', input_texts[seq_index])
        print('Decoded sentence:', decoded_sentence)
        
    print()

Train on 2228 samples, validate on 557 samples
Epoch 1/1
2228/2228 [==============================] - 31s 14ms/step - loss: 0.9196 - acc: 0.8083 - val_loss: 1.6485 - val_acc: 0.6883
--------------------------------------------------
Input sentence: Wow!
Decoded sentence: म                                                                                                                           
--------------------------------------------------
Input sentence: Help!
Decoded sentence: म                                                                                                                           
--------------------------------------------------
Input sentence: Jump.
Decoded sentence: म                                                                                                                           
--------------------------------------------------
Input sentence: Jump.
Decoded sentence: म                                                                               

In [14]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model( [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_dict = dict((i, char) for char, i in input_token_dict.items())
reverse_target_char_dict = dict((i, char) for char, i in target_token_dict.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, n_dec_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_dict['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_dict[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or len(decoded_sentence) > dec_seq_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, n_dec_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(25):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = enc_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-'*50)
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

--------------------------------------------------
Input sentence: Wow!
Decoded sentence: म                                                                                                                           
--------------------------------------------------
Input sentence: Help!
Decoded sentence: म                                                                                                                           
--------------------------------------------------
Input sentence: Jump.
Decoded sentence: म                                                                                                                           
--------------------------------------------------
Input sentence: Jump.
Decoded sentence: म                                                                                                                           
--------------------------------------------------
Input sentence: Jump.
Decoded sentence: म                                             